# Observabilité

## Surveiller sa consommation OVH avec Python, Prometheus et Grafana

On va parler :

* de supervision
* de base de séries temporelles / métriques - avec *Prometheus*
* de visualisation de données - avec *Grafana*
* des API OVH de suivi des consommations (Public Cloud)
* de l'extraction des métriques via un exporter - avec *Python*



# Base de séries temporelles - en général

## Fonctionnellement

* Des données **numériques**
* **datées**
* Associées à un nom de **métrique**, et des **étiquettes** / libellés
* Un langage de requêtage étendu (aggrégations, combinaisons)

**Une métrique**

```
# timestamp EPOCH
1716969870000
# métrique et labels
os_cpu_load{instance="loupo.infra.kobalt.fr",job="jmx-exporter",kobalt_client="infrastructure",kobalt_env="production"}
# valeur
0.0202991452991453
```

> *Taux d'occupation CPU* de *loupo.infra.kobalt.fr* le *mercredi 29 mai 2024 10:04:30 GMT+02:00 DST* : *2,02%*

**Une requête promQL**

```
sum(node_filesystem_avail_bytes{kobalt_client="infrastructure",device=~"/dev/sd.+"}) by (instance)
```

> Pour chaque serveur : somme des espaces occupés des systèmes de fichier

# Bases de séries temporelles

![tsdb.png](tsdb.png)

*source: Wikipedia https://en.wikipedia.org/wiki/Time_series_database (2024-05)*

# Base de séries temporelles - Prometheus

# Métriques et requêtage

* **Métriques**
  * Nom de métrique : alphanumérique `[a-zA-Z_:][a-zA-Z0-9_:]*`
  * 0..n labels alphanumériques
    * Nom : `[a-zA-Z_][a-zA-Z0-9_]*`
    * Valeur : Unicode
  * Trois types de métriques :
    * Jauge : valeur numérique arbitraire
    * Compteur : valeur numérique croissante
    * Histogramme : nombre d'appels, min, max, temps total, quantiles
  * Valeur : flottant 64 bits (IEEE 754 double-precision binary floating-point format)
  * Axe temporel : timestamp (granularité milliseconde)
* **Requêtage**
  * langage PromQL

*https://prometheus.io/docs/concepts/data_model/*

# Base de séries temporelles - Prometheus

## Architecture générale

* Collecte pull à périodicité constante
  * Via HTTP(s)
  * Authentification basic / oauth2 / certificats TLS
  * Alternative Push Gateway
* Un écosystème de collecte : les *exporters*
  * Serveur HTTP(s) légers
  * Implémentés en Go, Python, ...
  * node_exporter : informations systèmes, postgres_exporter : information serveur PostgreSQL, ...
* Interface basique de consultation
* Un mécanisme de notification
  * Des règles (métriques calculées) crée les alertes
  * Un composant *AlertManager* route les alertes


# Base de séries temporelles - Prometheus

![prometheus-ui.png](prometheus-ui.png)

# Visualisation - Grafana

## Pourquoi ?

* Prometheus permet le stockage et le requêtage
* Prometheus ne gère pas la visualisation
* Utilisation d'outils de visualisation

> Grafana, Kibana, ...

## Grafana

* Multi-source (150+)
  * SQL
  * Time-series
  * Et d'autres
* Conception de dashboard
  * Formulaires
  * Composants de visualisation (tableaux, graphes, ...)
  * Un outil de partage des dashboards
* Mécanisme de notification / alerte dédié

# Visualisation - Grafana

![grafana-ui.png](grafana-ui.png)


# Revenons à notre sujet

* Que veut-on surveiller ? consommation Public Cloud OVH
  * Nombre de CPU
  * Quantité de mémoire
  * Espace disque
  * Espace object storage
  * Bande passante object storage
  * Par projet
  * Par région
  * Par item
* Quelles sont les informations à disposition ? API OVH
  * /cloud/project/{service_id}
  * /cloud/project/{service_id}/quota
  * /cloud/project/{service_id}/instance
  * /cloud/project/{service_id}/storage
  * /cloud/project/{service_id}/usage/current
  * /cloud/project/{service_id}/volume
* Quelle mécanisme de collecte ? exporter Prometheus, écrit en Python
* Quelle visualisation ? un dashboard Grafana

# Pour résumer

* Des appels API pour récupérer les données
* Transcription dans le format Prometheus / Openmetrics
* Génération d'un endpoint HTTP(s)
* Récolte par Prometheus

# ovh_exporter

* https://github.com/kobalt/ovh_exporter
* Ecrit en Python
* 1 utilisateur (pour l'instant ?)

![ovh_exporter.png](ovh_exporter.png)

# ovh_exporter - Dépendances & structure

## Dépendances

```toml
dependencies = [
  "click~=8.1.6",          # Parsing de la ligne de commande
  "ovh~=1.1",              # Client OVH officiel
  "prometheus_client<1",   # Bibliothèque prometheus
  "PyYAML~=6.0",           # Parser YAML (configuration)
  "python-dotenv~=1.0",    # Support fichiers dotenv (configuration)
  "jsonschema",            # Validation Json Schema (configuration)
  "gunicorn"               # Container WSGI
]
```

## Structure

```
auth.py                    # Gestion des tokens d'API OVH
cli.py                     # Endpoints ligne de commande
collector.py               # Récolte et transcription des métriques
config.py                  # Parsing / validation de la configuration
logger.py                  # Initialisation logging
ovh_client.py              # Wrapper léger du client OVH
wsgi.py                    # Initialisation WSGI : middleware authentification + TLS
```

## Autres

* build : hatch
* tests : aucun
* Dockerfile

# ovh_exporter - Démonstration

* docker-compose :
  * Prometheus et sa configuration de collecte
  * Grafana et son dashboard
* Injection 2 semaines de données
* Configuration d'ovh_exporter
* Obtention des crédentials API
* Démarrage d'ovh_exporter
* Visualisation des résultats : exporter / Prometheus / Grafana

## Docker-compose

```yaml
version: '2.1'

services:
  prometheus:
    image: prom/prometheus:latest
    container_name: prometheus
    volumes:
    - ./prometheus:/etc/prometheus
    - prometheus-data:/prometheus
    command:
    - '--config.file=/etc/prometheus/prometheus.yml'
    - '--enable-feature=exemplar-storage'
    expose:
    - 9090
    ports:
    - "9090:9090"
  grafana:
    image: grafana/grafana-oss
    container_name: grafana
    volumes:
    - ./grafana/provisioning:/etc/grafana/provisioning
    environment:
    - GF_SECURITY_ADMIN_USER=${ADMIN_USER:-admin}
    - GF_SECURITY_ADMIN_PASSWORD=${ADMIN_PASSWORD:-admin}
    - GF_USERS_ALLOW_SIGN_UP=false
    - GF_INSTALL_PLUGINS=grafana-clock-panel, grafana-simple-json-datasource
    ports:
    - 3000:3000

volumes:
  prometheus-data: {}
```

## ovh_exporter

### Configuration

```yaml
---
env_file: auth.env
server:
  bind_addr: 0.0.0.0
  port: 9100
  basic_auth:
    enabled: false
  tls:
    enabled: false
ovh:
  endpoint: ovh-eu
  application_key: ${APPLICATION_KEY}
  application_secret: ${APPLICATION_SECRET}
  #consumer_key: ${CONSUMER_KEY}
services:
- id: 1ac3fc0e525b4a1d8624bbc847f5414c
  labels:
    kobalt_client: infrastructure
    ovh_project: kobalt-sandbox
- id: e662561cee2347cf9e30d0dbb4c23168
  labels:
    kobalt_client: infrastructure
    ovh_project: infrastructure
```

### Initialisation du secret

```
hatch run ovh_exporter login
```

> Report dans le fichier `auth.env`

L'utilisation de la commande `login` facilite la configuration d'une clé strictement limitée aux services nécessaires :
* Limitation des endpoints
* Limitation à la méthode HTTP GET

Il est sinon possible de récupérer une clé « manuellement ».

### Lancement ovh_exporter

```
hatch run ovh_exporter -v debug server
```

## Consultation

* Consultation des métriques sur l'exporter : http://0.0.0.0:9100/metrics
* Consultation des métriques sur prometheus
  * Etat du endpoint : http://localhost:9090/targets?search=
  * Affichage de toutes les métriques sur les 15 dernières minutes : http://localhost:9090/graph?g0.expr=%7B__name__%3D~%22ovh.%2B%22%7D&g0.tab=0&g0.display_mode=lines&g0.show_exemplars=0&g0.range_input=15m
* Consultation des métriques sur Grafana
  * http://localhost:3000/d/a8aefc65-6d67-448a-a949-32aa55b1f9a7/ovh-exporter?orgId=1&from=now-15m&to=now

# Grafana - tableau de bord

![grafana-ui-ovh_exporter.png](grafana-ui-ovh_exporter.png)

# ovh_exporter - Extraits choisis

## prometheus_client - initialisation

```python

    # pylint: disable=too-many-statements
    def __init__(self, labelnames):
        # Storage
        storage_usage_labels = ["service_id", "region", "flavor"]
        self.ovh_usage_storage_gb_hours = GaugeMetricFamily(
            "ovh_usage_storage_gb_hours",
            "Storage usage in gb x hours",
            labels=labelnames + storage_usage_labels,
        )
        self.ovh_usage_storage_price = GaugeMetricFamily(
            "ovh_usage_storage_price",
            "Storage usage price",
            labels=labelnames + storage_usage_labels,
        )
        # [...] + ~30 autres métriques
```

## prometheus_client - collecte d'une métrique

```python
    def _collect_volumes(self, metrics: Metrics, service, volumes):
        """Collect volume information."""
        for volume in volumes:
            try:
                gauge_value = int(volume["size"])
                metrics.ovh_volume_size_gb.add_metric(
                    self._labels(service, [
                        service.id,
                        volume["id"],
                        volume["name"],
                        volume["region"],
                        volume["type"],
                    ]),
                    gauge_value,
                )
            except (TypeError, ValueError):
                log.warning("Volume %s ignored as size is missing", volume["id"])
```

## WSGI - TLS et basic auth

Utilisation des API Gunicorn et mise en oeuvre d'un middleware d'authentification.

**Configuration serveur et stack TLS**

```python
class StandaloneApplication(gunicorn.app.base.BaseApplication):
    # 'init' and 'load' methods are implemented by WSGIApplication.
    # pylint: disable=abstract-method
    """Gunicorn wrapper."""
    def __init__(self, app, bind_addr="127.0.0.1", bind_port=9100,
                 cert_file=None, key_file=None):
        self.cert_file = cert_file
        self.key_file = key_file
        self.bind_addr = bind_addr
        self.bind_port = bind_port
        self.application = app
        super().__init__()

    def load_config(self):
        config = {}
        if self.cert_file and self.key_file:
            config["certfile"] = self.cert_file
            config["keyfile"] = self.key_file
        config["bind"] = f"{self.bind_addr}:{self.bind_port}"
        config["workers"] = 3
        config["worker_class"] = "gthread"
        config["timeout"] = 180
        for key, value in config.items():
            self.cfg.set(key.lower(), value)

    def load(self):
        return self.application
```

**Middleware chargé de l'authentification Basic**

```python
class BasicAuthMiddleware:
    """Basic Auth WSGI middleware"""
    def __init__(self, app, login, password, realm="ovh_exporter"):
        self.app = app
        self.realm = realm
        self.login = login
        self.password = password

    def __call__(self, environ, start_response):
        if not self._check_auth(environ):
            start_response(
                "401 Unauthorized",
                [("WWW-Authenticate", f"Basic realm={self.realm}, charset=\"UTF-8\"")])
            return []
        return self.app(environ, start_response)

    def _check_auth(self, environ):
        authorization = environ.get("HTTP_AUTHORIZATION", None)
        if authorization:
            auth = self._extract_authorization(authorization)
            if auth and self.login == auth[0] and self.password == auth[1]:
                return True
        return False

    def _extract_authorization(self, authorization_header: str):
        """Extract decoded Authorization."""
        if not authorization_header.startswith("Basic "):
            log.debug("Authorization is not a basic authentication.")
            return False
        try:
             # Strip 'Basic '
            authorization_base64 = authorization_header[6:].encode('ascii')
            auth = base64.decodebytes(authorization_base64).decode('utf-8')
            separator = auth.find(":")
            if separator != -1:
                return (auth[0:separator], auth[separator+1:])
            else: # noqa: RET505
                log.debug("Decoded authorization cannot be parsed.")
                return False
        except binascii.Error:
            log.debug("Authorization is not a basic authentication.")
            return False
```

# Métriques disponibles

> cf README.md

## Labels

* Ventilation systématique par
  * projet / service
  * region

## Métriques

* object storage
  * storage consumption / price (gb.hour) (2 metrics)
  * external / external - inconming / outgoing bandwidth - consumption / price (8 metrics)
* volumes
  * size (gb)
* quota (current / max)
  * volume size
  * volume backup
  * volume count
  * volume backup count
  * instance count
  * cpu count
  * ram (gb)
  * network count
  * network subnet count
  * network floating ip count
  * network gateway count
  * load balancer count
  * key manager count
* storage
  * size
  * object count
* instance usage
  * hours
  * price
* volume
  * gb x hours consumption
  * price

# Autre considérations

## Prometheus

* Paramétrage d'alertes
* Des trous dans les graphes...

## Grafana

* Autres ventilations (les labels sont déjà disponibles)
  * par instance
  * par périphériques blocs
  * par bucket object storage
  * ...

## ovh_exporter

* Utilisation d'un cache
* Filtrer les métriques
* Filtrer les régions
* Ignorer les régions vides
* Multi-target prometheus : une requête par projet
* Autres mécanismes d'authentification
* Bascule API OVH sur l'authentification oauth2

## prometheus_client

* Expose automatique des métriques python : https://prometheus.github.io/client_python/collector/
* Instrumentation directe d'une application

# Merci à tous

Best viewed with `~/.jupyter/custom/custom.css`


```css
.jp-RenderedHTMLCommon {
  min-height: 600px; !important
}

.jp-Cell {
  opacity: 0.33;
}

.jp-Cell.jp-mod-active {
  opacity: 1;
}
```